In [67]:
import os
import gdal
import matplotlib.pyplot as plt
import geopandas as gpd
from rasterio import features, coords, plot
import rasterio
from shapely.geometry import shape
from geopy import distance
import skimage.io as ski
%matplotlib inline
# https://gis.stackexchange.com/questions/151339/rasterize-a-shapefile-with-geopandas-or-fiona-python
ROOT_DIR = "/home/rave/deeplearn_imagery/data/raw/stephtest/"
os.chdir(ROOT_DIR)
WV2_LABEL_DIR = 'merged_digitized_labels_raster_extents/'
WV2_IMAGERY_DIR = 'projectedtiffs/'
RASTERIZED_DIR = 'rasterized_wv2_labels/'
NEGBUFF = "-.5"

def list_paths(directory, suffix):
    # all files, only the projected images
    items = os.listdir(directory)

    merged_list = []
    for name in items:
        if name.endswith(suffix):
            merged_list.append(directory+name)
    return merged_list

merged_list = list_paths(WV2_IMAGERY_DIR, '.shp')

def takefirst(elem):
    return int(elem[-10])

shp_list = sorted(merged_list, key=takefirst)

# list the projected images, just growing season
scenes = os.listdir(WV2_IMAGERY_DIR)

merged_list = []
for name in scenes:
    if "GS" in name and "xml" not in name:
        merged_list.append(WV2_IMAGERY_DIR+name)  

def takefirst_two(elem):
    return int(elem[-12:-10])

img_list = sorted(merged_list, key=takefirst_two)
print(img_list)

for shp_path, img_path in zip(shp_list, img_list):
    
    shp_frame = gpd.read_file(shp_path)
    
    with rasterio.open(img_path) as rast:
        meta = rast.meta.copy()
        meta.update(compress="lzw")
        meta['count'] = 1
        
    rasterized_name = img_path[-12:-4]+"_label.tif"
    with rasterio.open(RASTERIZED_DIR+rasterized_name, 'w', **meta) as out:
            out_arr = out.read(1)
            # this is where we create a generator of geom, value pairs to use in rasterizing
            shp_frame['DN'].iloc[0] = 0
            shp_frame['DN'].iloc[1:] = 1
            maxx_bound = shp_frame.bounds.maxx.max()
            minx_bound = shp_frame.bounds.minx.min()
            if maxx_bound >= 30 and minx_bound>= 30:
                shp_frame = shp_frame.to_crs({'init': 'epsg:32736'})
                shp_frame['geometry'] = shp_frame['geometry'].buffer(float(NEGBUFF))
                shp_frame = shp_frame.to_crs({'init': 'epsg:4326'})
                                                  
            else:
                shp_frame = shp_frame.to_crs({'init': 'epsg:32735'})
                shp_frame['geometry'] = shp_frame['geometry'].buffer(float(NEGBUFF))
                shp_frame = shp_frame.to_crs({'init': 'epsg:4326'})
            
            # hacky way of getting rid of empty geometries
            shp_frame = shp_frame[shp_frame.Shape_Area > 9e-11]
            shapes = ((geom,value) for geom, value in zip(shp_frame.geometry, shp_frame.DN))

            burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
            out.write_band(1, burned)
            
# all files, only the projected images
rasts = list_paths(RASTERIZED_PATH, '.tif')        
for rast in rasts:
    
    arr = ski.imread(rast)
    arr[arr ==  arr.min()] = 0
    ski.imsave(rast, arr)

['projectedtiffs/01_MS_GS.tif', 'projectedtiffs/02_MS_GS.tif', 'projectedtiffs/03_MS_GS.tif', 'projectedtiffs/04_MS_GS.tif', 'projectedtiffs/05_MS_GS.tif', 'projectedtiffs/06_MS_GS.tif', 'projectedtiffs/07_MS_GS.tif', 'projectedtiffs/08_MS_GS.tif']
